In [135]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import svm, metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [141]:
#x = pd.read_csv("CT_Farmington.csv")
x = pd.read_csv("WI_BlackEarthCreek.csv")
xx = x.copy()[[c for c in x.columns if c not in ['region','site']]] \
    .set_index(pd.to_datetime(x['DateTime_UTC'])).resample("15Min").mean()
xx = xx[xx.index>="2010-10-02"]
variables = xx.columns.tolist()
print variables

# Create flags, assign first value to be flagged
xx['flag'] = 0

var = 'DO_mgL' # variable to add error

frac = 0.20 # fraction of maximum error
dayn = 30 # das of training data
obsn = 14 # days of testing data
outall = pd.DataFrame()


['DO_mgL', 'Discharge_m3s', 'SpecCond_uScm', 'WaterTemp_C', 'pH']


In [142]:
# set up k-folds
# break into 180 day chunks
alldays = np.sort(list(set(xx.index.date)))
breaks = np.array_split(alldays, int(len(alldays)/180))

# fraction errors to introduce
fracs = [0.05,0.1,0.15,0.2]

In [143]:
for frac in fracs:
    output = pd.DataFrame(columns=['days','accuracy'])
    print frac

    for k in xrange(len(breaks)):
        xk = xx.copy().iloc[(xx.index.date > breaks[k][0]) & (xx.index.date <= breaks[k][-1])]
        print k
        
        drange = xrange(7,180,30)
        for dayn in drange:
            train = xk.copy()[xk.index <= xk.index[0]+timedelta(days=dayn)]
            test = xk.copy()[(xk.index > xk.index[0]+timedelta(days=dayn)) & (xk.index <= xk.index[0]+timedelta(days=dayn+obsn))]

            #clf = svm.LinearSVC(C=1e5)
            xtrain = train.dropna()
            if dayn==drange[0]:
                clf = svm.SVC(C=1000)
                xtrain.loc[xtrain.index[0],'flag'] = 1
            else:
                clf = svm.NuSVC(nu=0.05)
            xsvm = xtrain.as_matrix(variables)
            scaler = StandardScaler().fit(xsvm)
            clf.fit(scaler.transform(xsvm), xtrain.flag)

            # random error assignment - half of two week sample
            xtest = test.copy().dropna()
            werr = np.random.choice(range(0, len(xtest)), len(xtest)/2, False)
            xtest.iloc[werr,xtest.columns==var] = xtest.iloc[werr,xtest.columns==var]*(1-frac)
            xtest['actual'] = 0
            xtest.iloc[werr,xtest.columns=='actual'] = 1 # which have error added

            xpred = xtest.as_matrix(variables)
            xtest['pred'] = clf.predict(scaler.transform(xpred)).tolist() # -1 are anomalies

            xk.loc[xtest.index, 'flag'] = xtest['actual'] #replace error data, perfect detection
            xk.loc[xtest.index, var] = xtest[var] # replace variable data for update

            #print str(dayn)+": "+str(len(xtest))+"/"+str(len(xtrain))
            outacc = metrics.accuracy_score(xtest['actual'], xtest['pred'])
            output = pd.concat([output, pd.DataFrame([[dayn,outacc]], columns=['days','accuracy'])])

    output['frac'] = frac
    outall = pd.concat([outall, output])


0.05
0
1
2
3
4
5
0.1
0
1
2
3
4
5
0.15
0
1
2
3
4
5
0.2
0
1
2
3
4
5


In [146]:
outall.to_csv("error_checking_output.csv",index=False)
